# Segmentation and Clustering of neighborhoods in Toronto

Lets import beautiful soup, requests library and pandas

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Now read html and store it in a variable

In [2]:
html_source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(html_source,'lxml')

In the following code, we would use beautifulsoup library to fetch text inside each tr block, assign each td block into variables named postcode, borough and neighborhood.
A list of dictionaries is used to store initial data
This time, We will not use comma separated neighborhood data as this will result in poor clustering and segmentation.

Finally creating the intended pandas dataframe we need.

In [3]:
table_text=soup.find('table', class_="wikitable")
row_data=table_text.find_all('tr')
#row_data
#row_data.td.text
rows_list=[]
postNeighborDict={}
row_index=0
for row_data in table_text.find_all('tr'):
    cell_datas=row_data.find_all('td')
    if len(cell_datas)==3:
        postcode=cell_datas[0].text
        borough=cell_datas[1].text
        neighborhood=cell_datas[2].text
        neighborhood=neighborhood.replace('\n','')
        #print(postcode, "==", borough, "--", neighborhood)
        if borough!="Not assigned":
            if neighborhood=="Not assigned":
                neighborhood=borough
            dict={'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood}
            postNeighborDict[postcode]=row_index
            rows_list.append(dict)
            row_index=row_index+1
postalcode_df=pd.DataFrame(rows_list,columns=['PostalCode','Borough','Neighborhood'])

In [4]:
postalcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Let's print the size of this dataframe

In [5]:
print(postalcode_df.shape)

(212, 3)


Let's create a new dataframe postalcode_df2 having two new columns Latitude and Longitude

In [6]:
postalcode_df2=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude'])
postalcode_df2[['PostalCode','Borough','Neighborhood']]=postalcode_df[['PostalCode','Borough','Neighborhood']]


Now GeoCode api is not working in my case, Hence let's download csv file containing locations.

In [7]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('file downloaded')

file downloaded


Read csv file, Store the data in pandas dataframe  

In [8]:
geospatial_df = pd.read_csv('Geospatial_data.csv')
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's fetch latitude longitude on the basis of postal code

In [16]:
for index_value in range(212):
    latlng_value=geospatial_df.loc[ geospatial_df['Postal Code'] == postalcode_df2.loc[index_value]['PostalCode']]
    latitude=latlng_value.iloc[0]['Latitude']
    longitude=latlng_value.iloc[0]['Longitude']
    postalcode_df2.loc[index_value]['Latitude']=latitude
    postalcode_df2.loc[index_value]['Longitude']=longitude
    #print("index_value : ",index_value, "Latitude: ", latitude, ": Longitude", longitude)

The complete dataframe we obtain is :

In [18]:
postalcode_df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606
4,M6A,North York,Lawrence Heights,43.7185,-79.4648


Now, we will follow Segmentationa and Clustering of Newyork city example for the final task.
Step 1: - Import Folium library

In [19]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Library imported')

Library imported


### Import Geopy library

In [20]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Geopy library imported')

Geopy library imported


#### Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [21]:
address = 'Toronto City, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7394839, -79.369314.


#### Create a map of Toronto with neighborhoods superimposed on top

In [22]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(postalcode_df2['Latitude'], postalcode_df2['Longitude'], postalcode_df2['Borough'], postalcode_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We will use borough containing word 'Toronto' for our analysis. So let's slice our data and create another new dataframe for our analysis

In [23]:
condition_check=postalcode_df2['Borough'].str.contains('Toronto')
toronto_data = postalcode_df2[condition_check].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
1,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3789
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3789
4,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754


#### Let's recreate map of toronto with this data

In [24]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'TR4OJXMM340Z5YTVDZ1QGAG1YHPBOJ4NEPK4V52K10Y0RYPY' # your Foursquare ID
CLIENT_SECRET = 'DZFZHGTHTBEWNOM25GNI24LBCGGUTLVPQMALSZQBQQWSIZMT' # your Foursquare Secret
VERSION = '20190315' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TR4OJXMM340Z5YTVDZ1QGAG1YHPBOJ4NEPK4V52K10Y0RYPY
CLIENT_SECRET:DZFZHGTHTBEWNOM25GNI24LBCGGUTLVPQMALSZQBQQWSIZMT


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [26]:
toronto_data.loc[0, 'Neighborhood']

'Harbourfront'

Get the neighborhood's latitude and longitude values.

In [27]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.

First, let's create the GET request URL.

In [28]:
# type your answer here
LIMIT=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT
)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TR4OJXMM340Z5YTVDZ1QGAG1YHPBOJ4NEPK4V52K10Y0RYPY&client_secret=DZFZHGTHTBEWNOM25GNI24LBCGGUTLVPQMALSZQBQQWSIZMT&v=20190315&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the resutls

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8e6a3edd57973aaf854e67'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Let's borrow the **get_category_type** function from the Foursquare lab.

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's clean json and convert it into pandas dataframe

In [31]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now lets write the code to run the above function on each neighborhood and create a new dataframe.

In [34]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(3268, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
Brockton,20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,14,14,14,14,14,14
Cabbagetown,44,44,44,44,44,44
Central Bay Street,82,82,82,82,82,82
Chinatown,98,98,98,98,98,98


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyze Each Neighborhood

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [39]:
toronto_onehot.shape

(3268, 237)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,CN Tower,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010204,0.000000,0.00,0.051020,0.000000,0.051020,0.010204,0.000000,0.000000,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [41]:
toronto_grouped.shape

(73, 237)

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2  Thai Restaurant  0.04
3              Bar  0.04
4       Steakhouse  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3          Boutique  0.07
4           Airport  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----Brockton----
            venue  freq
0     Coffee Shop  0.10
1  Breakfast Spot  0.10
2            Café  0.10
3             Gym  0.05
4         Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1                 Spa  0.06
2          Restaurant  0.06
3       Burrito Place  0.06
4             Butcher  0.06


----CN Tower----
              venue  freq
0    Airport Lounge  0.14
1

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [44]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Bakery,Burger Joint,Gym,Sushi Restaurant,Asian Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
2,Berczy Park,Coffee Shop,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Steakhouse,Bakery,Italian Restaurant,Restaurant
3,Brockton,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store,Convenience Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Garden,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [45]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 2, 2, 2, 3, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606,2,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
1,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606,2,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3789,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Diner,Pizza Place,Italian Restaurant,Japanese Restaurant
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3789,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Diner,Pizza Place,Italian Restaurant,Japanese Restaurant
4,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,2,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Hotel,Pizza Place,Gastropub,Clothing Store,Italian Restaurant


In [47]:
toronto_merged['Cluster Labels'].value_counts()

2    61
3     7
0     4
1     2
Name: Cluster Labels, dtype: int64

Finally, let's visualize the resulting clusters

In [48]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,0,Park,Swim School,Bus Line,Women's Store,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
34,Central Toronto,0,Trail,Jewelry Store,Park,Sushi Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
35,Central Toronto,0,Trail,Jewelry Store,Park,Sushi Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
66,Downtown Toronto,0,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 2

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,1,Playground,Gym,Tennis Court,Restaurant,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
50,Central Toronto,1,Playground,Gym,Tennis Court,Restaurant,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


#### Cluster 3

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
1,Downtown Toronto,2,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Diner,Pizza Place,Italian Restaurant,Japanese Restaurant
3,Downtown Toronto,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Diner,Pizza Place,Italian Restaurant,Japanese Restaurant
4,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Hotel,Pizza Place,Gastropub,Clothing Store,Italian Restaurant
5,East Toronto,2,Coffee Shop,Health Food Store,Pub,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
6,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Steakhouse,Bakery,Italian Restaurant,Restaurant
7,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Salad Place,Sushi Restaurant,Chinese Restaurant,Thai Restaurant
8,Downtown Toronto,2,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Coffee Shop
9,Downtown Toronto,2,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Bakery,Burger Joint,Gym,Sushi Restaurant,Asian Restaurant


#### Cluster 4

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
60,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
61,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
62,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
63,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
64,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
65,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane


This shows the distinct clusters and similarity in them.